# Enable PII detection in tests

Learn how to enable and configure Personally Identifiable Information (PII) detection when running tests with the ValidMind Library. Choose whether or not to include PII in test descriptions generated, or whether or not to include PII in test results logged to the ValidMind Platform.

::: {.content-hidden when-format="html"}
## Contents    
- [About ValidMind](#toc1__)    
  - [Before you begin](#toc1_1__)    
  - [New to ValidMind?](#toc1_2__)    
  - [Key concepts](#toc1_3__)    
- [Setting up](#toc2__)    
  - [Install the ValidMind Library with PII detection](#toc2_1__)    
  - [Initialize the ValidMind Library](#toc2_2__)    
    - [Get your code snippet](#toc2_2_1__)    
- [Using PII detection](#toc3__)    
  - [Create a custom test that outputs PII](#toc3_1__)    
  - [Run test under different PII detection modes](#toc3_2__)    
    - [disabled](#toc3_2_1__)    
    - [test_results](#toc3_2_2__)    
    - [test_descriptions](#toc3_2_3__)    
    - [all](#toc3_2_4__)    
  - [Override detection](#toc3_3__)    
    - [Override test result logging](#toc3_3_1__)    
    - [Override test descriptions and test result logging](#toc3_3_2__)    
  - [Review logged test results](#toc3_4__)    
- [Troubleshooting](#toc4__)    
- [Learn more](#toc5__)    

:::
<!-- jn-toc-notebook-config
	numbering=false
	anchor=true
	flat=false
	minLevel=2
	maxLevel=4
	/jn-toc-notebook-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

<a id='toc1__'></a>

## About ValidMind

ValidMind is a suite of tools for managing model risk, including risk associated with AI and statistical models. 

You use the ValidMind Library to automate documentation and validation tests, and then use the ValidMind Platform to collaborate on model documentation. Together, these products simplify model risk management, facilitate compliance with regulations and institutional standards, and enhance collaboration between yourself and model validators.

<a id='toc1_1__'></a>

### Before you begin

This notebook assumes you have basic familiarity with Python, including an understanding of how functions work. If you are new to Python, you can still run the notebook but we recommend further familiarizing yourself with the language. 

If you encounter errors due to missing modules in your Python environment, install the modules with `pip install`, and then re-run the notebook. For more help, refer to [Installing Python Modules](https://docs.python.org/3/installing/index.html).

<a id='toc1_2__'></a>

### New to ValidMind?

If you haven't already seen our documentation on the [ValidMind Library](https://docs.validmind.ai/developer/validmind-library.html), we recommend you begin by exploring the available resources in this section. There, you can learn more about documenting models and running tests, as well as find code samples and our Python Library API reference.

<div class="alert alert-block alert-info" style="background-color: #B5B5B510; color: black; border: 1px solid #083E44; border-left-width: 5px; box-shadow: 2px 2px 4px rgba(0, 0, 0, 0.2);border-radius: 5px;"><span style="color: #083E44;"><b>For access to all features available in this notebook, you'll need access to a ValidMind account.</b></span>
<br></br>
<a href="https://docs.validmind.ai/guide/configuration/register-with-validmind.html" style="color: #DE257E;"><b>Register with ValidMind</b></a></div>

<a id='toc1_3__'></a>

### Key concepts

**Model documentation**: A structured and detailed record pertaining to a model, encompassing key components such as its underlying assumptions, methodologies, data sources, inputs, performance metrics, evaluations, limitations, and intended uses. It serves to ensure transparency, adherence to regulatory requirements, and a clear understanding of potential risks associated with the model’s application.

**Documentation template**: Functions as a test suite and lays out the structure of model documentation, segmented into various sections and sub-sections. Documentation templates define the structure of your model documentation, specifying the tests that should be run, and how the results should be displayed.

**Tests**: A function contained in the ValidMind Library, designed to run a specific quantitative test on the dataset or model. Tests are the building blocks of ValidMind, used to evaluate and document models and datasets, and can be run individually or as part of a suite defined by your model documentation template.

**Metrics**: A subset of tests that do not have thresholds. In the context of this notebook, metrics and tests can be thought of as interchangeable concepts.

**Custom metrics**: Custom metrics are functions that you define to evaluate your model or dataset. These functions can be registered with the ValidMind Library to be used in the ValidMind Platform.

**Inputs**: Objects to be evaluated and documented in the ValidMind Library. They can be any of the following:

  - **model**: A single model that has been initialized in ValidMind with [`vm.init_model()`](https://docs.validmind.ai/validmind/validmind.html#init_model).
  - **dataset**: Single dataset that has been initialized in ValidMind with [`vm.init_dataset()`](https://docs.validmind.ai/validmind/validmind.html#init_dataset).
  - **models**: A list of ValidMind models - usually this is used when you want to compare multiple models in your custom metric.
  - **datasets**: A list of ValidMind datasets - usually this is used when you want to compare multiple datasets in your custom metric. (Learn more: [Run tests with multiple datasets](https://docs.validmind.ai/notebooks/how_to/run_tests_that_require_multiple_datasets.html))

**Parameters**: Additional arguments that can be passed when running a ValidMind test, used to pass additional information to a metric, customize its behavior, or provide additional context.

**Outputs**: Custom metrics can return elements like tables or plots. Tables may be a list of dictionaries (each representing a row) or a pandas DataFrame. Plots may be matplotlib or plotly figures.

**Test suites**: Collections of tests designed to run together to automate and generate model documentation end-to-end for specific use-cases.

Example: the [`classifier_full_suite`](https://docs.validmind.ai/validmind/validmind/test_suites/classifier.html#ClassifierFullSuite) test suite runs tests from the [`tabular_dataset`](https://docs.validmind.ai/validmind/validmind/test_suites/tabular_datasets.html) and [`classifier`](https://docs.validmind.ai/validmind/validmind/test_suites/classifier.html) test suites to fully document the data and model sections for binary classification model use-cases.

<a id='toc2__'></a>

## Setting up

<a id='toc2_1__'></a>

### Install the ValidMind Library with PII detection

<div class="alert alert-block alert-info" style="background-color: #B5B5B510; color: black; border: 1px solid #083E44; border-left-width: 5px; box-shadow: 2px 2px 4px rgba(0, 0, 0, 0.2);border-radius: 5px;"><span style="color: #083E44;"><b>Recommended Python versions</b></span>
<br></br>
Python 3.8 <= x <= 3.11</div>

To use PII detection powered by [Microsoft Presidio](https://microsoft.github.io/presidio/), install the library with the explicit `[pii-detection]` extra specifier:

In [ ]:
%pip install -q "validmind[pii-detection]"

<a id='toc2_2__'></a>

### Initialize the ValidMind Library

ValidMind generates a unique _code snippet_ for each registered model to connect with your developer environment. You initialize the ValidMind Library with this code snippet, which ensures that your documentation and tests are uploaded to the correct model when you run the notebook.

<a id='toc2_2_1__'></a>

#### Get your code snippet

1. In a browser, [log in to ValidMind](https://docs.validmind.ai/guide/configuration/log-in-to-validmind.html).

2. In the left sidebar, navigate to **Inventory** and click **+ Register Model**.

3. Enter the model details and click **Continue**. ([Need more help?](https://docs.validmind.ai/guide/model-inventory/register-models-in-inventory.html))

4. Go to **Getting Started** and click **Copy snippet to clipboard**.

Next, [load your model identifier credentials from an `.env` file](https://docs.validmind.ai/developer/model-documentation/store-credentials-in-env-file.html) or replace the placeholder with your own code snippet:

In [ ]:
# Load your model identifier credentials from an `.env` file

%load_ext dotenv
%dotenv .env

# Or replace with your code snippet

import validmind as vm

vm.init(
    # api_host="...",
    # api_key="...",
    # api_secret="...",
    # model="...",
)

<a id='toc3__'></a>

## Using PII detection

<a id='toc3_1__'></a>

### Create a custom test that outputs PII

To demonstrate the feature, we'll need a test that outputs PII. First we'll create a custom test that returns:

- A description string containing PII (name, email, phone)
- A small table containing PII in columns

This output mirrors the structure used in other custom test notebooks and will exercise both table and description PII detection paths. However, if structured detection is unavailable, the library falls back to token-level text scans when possible.

In [ ]:
import pandas as pd

from validmind import test

@test("pii_demo.PIIDetection")
def pii_custom_test():
    """A custom test that returns demo PII.
    This default test description will display when PII is not sent to the LLM to generate test descriptions based on test result data."""
    return pd.DataFrame(
        {
            "name": ["Jane Smith", "John Doe", "Alice Johnson"],
            "email": [
                "jane.smith@bank.example",
                "john.doe@company.example",
                "alice.johnson@service.example",
            ],
            "phone": ["(212) 555-9876", "(415) 555-1234", "(646) 555-5678"],
        }
    )

<div class="alert alert-block alert-info" style="background-color: #B5B5B510; color: black; border: 1px solid #083E44; border-left-width: 5px; box-shadow: 2px 2px 4px rgba(0, 0, 0, 0.2);border-radius: 5px;"><span style="color: #083E44;"><b>Want to learn more about custom tests?</b></span>
<br></br>
Check out our extended introduction to custom tests — <a href="https://docs.validmind.ai/notebooks/code_samples/custom_tests/implement_custom_tests.html" style="color: #DE257E;"><b>Implement custom tests</b></a></div>

<a id='toc3_2__'></a>

### Run test under different PII detection modes

Next, let's import [the `run_test` function](https://docs.validmind.ai/validmind/validmind/tests.html#run_test) provided by the `validmind.tests` module to run our custom test via a function called `run_pii_test()` that catches exceptions to observe blocking behavior when PII is present:

In [ ]:
import os
from validmind.tests import run_test

# Run test and tag result with unique `result_id`
def run_pii_test(result_id=""):
    try:
        test_name = f"pii_demo.PIIDetection:{result_id}"
        result = run_test(test_name)

        # Check if the test description was generated by LLM
        if not result._was_description_generated:
            print("PII detected: LLM-generated test description skipped")
        else:
            print("No PII detected or detection disabled: Test description generated by LLM")

        # Try logging test results to the ValidMind Platform
        result.log()
        print("No PII detected or detection disabled: Test results logged to the ValidMind Platform")
    except Exception as e:
        print("PII detected: Test results not logged to the ValidMind Platform")

We'll then switch the `VALIDMIND_PII_DETECTION` environment variable across modes in the below examples.

<div class="alert alert-block alert-info" style="background-color: #B5B5B510; color: black; border: 1px solid #083E44; border-left-width: 5px; box-shadow: 2px 2px 4px rgba(0, 0, 0, 0.2);border-radius: 5px;"><span style="color: #083E44;"><b>Note that since we are running a custom test that does not exist in your model's default documentation template, we'll receive output indicating that a test-driven block doesn't currently exist in your model's documentation for that particular test ID.</b></span>
<br></br>
That's expected, as when we run custom tests the results logged need to be manually added to your documentation within the ValidMind Platform or added to your documentation template.</div>

<a id='toc3_2_1__'></a>

#### disabled

When detection is set to `disabled`, tests run and generate test descriptions. Logging tests with [`.log()`](https://docs.validmind.ai/validmind/validmind/vm_models.html#TestResult.log) will also send test descriptions and test results to the ValidMind Platform as usual:

In [ ]:
print("\n=== Mode: disabled ===")
os.environ["VALIDMIND_PII_DETECTION"] = "disabled"

# Run test and tag result with unique ID `disabled`
run_pii_test("disabled")

<a id='toc3_2_2__'></a>

#### test_results

When detection is set for `test_results`, tests run and generate test descriptions for review in your environment, but logging tests will not send descriptions or test results to the ValidMind Platform:

In [ ]:
print("\n=== Mode: test_results ===")
os.environ["VALIDMIND_PII_DETECTION"] = "test_results"

# Run test and tag result with unique ID `results_blocked`
run_pii_test("results_blocked")

<a id='toc3_2_3__'></a>

#### test_descriptions

When detection is set for `test_descriptions`, tests run but will not generate test descriptions, and logging tests will not send descriptions but will send test results to the ValidMind Platform:

In [ ]:
print("\n=== Mode: test_descriptions ===")
os.environ["VALIDMIND_PII_DETECTION"] = "test_descriptions"

# Run test and tag result with unique ID `desc_blocked`
run_pii_test("desc_blocked")

<a id='toc3_2_4__'></a>

#### all

When detection is set to `all`, tests run will not generate test descriptions or log test results to the ValidMind Platform.

In [ ]:
print("\n=== Mode: all ===")
os.environ["VALIDMIND_PII_DETECTION"] = "all"

# Run test and tag result with unique ID `all_blocked`
run_pii_test("all_blocked")

<a id='toc3_3__'></a>

### Override detection

You can override blocking by passing `unsafe=True` to `result.log(unsafe=True)`, but this is not recommended outside controlled workflows.

To demonstrate, let's rerun our custom test with some override scenarios.

<a id='toc3_3_1__'></a>

#### Override test result logging

First, let's rerun our custom test with detection set to `all`, which will send the test results but not the test descriptions to the ValidMind Platform:

In [ ]:
print("\n=== Mode: all & unsafe=True ===")
os.environ["VALIDMIND_PII_DETECTION"] = "all"

# Run test and tag result with unique ID `override_results`
try:
    result = run_test("pii_demo.PIIDetection:override_results")

    # Check if the test description was generated by LLM
    if not result._was_description_generated:
        print("PII detected: LLM-generated test description skipped")
    else:
        print("No PII detected or detection disabled: Test description generated by LLM")

    # Try logging test results to the ValidMind Platform
    result.log(unsafe=True)
    print("No PII detected, detection disabled, or override set: Test results logged to the ValidMind Platform")
except Exception as e:
    print("PII detected: Test results not logged to the ValidMind Platform")

<a id='toc3_3_2__'></a>

#### Override test descriptions and test result logging

To send both the test descriptions and test results via override, set the `VALIDMIND_PII_DETECTION` environment variable to `test_results` while including the override flag:

In [ ]:
print("\n=== Mode: test_results & unsafe=True ===")
os.environ["VALIDMIND_PII_DETECTION"] = "test_results"

# Run test and tag result with unique ID `override_both`
try:
    result = run_test("pii_demo.PIIDetection:override_both")

    # Check if the test description was generated by LLM
    if not result._was_description_generated:
        print("PII detected: LLM-generated test description skipped")
    else:
        print("No PII detected, detection disabled, or override set: Test description generated by LLM")

    # Try logging test results to the ValidMind Platform
    result.log(unsafe=True)
    print("No PII detected, detection disabled, or override set: Test results logged to the ValidMind Platform")
except Exception as e:
    print("PII detected: Test results not logged to the ValidMind Platform")

<a id='toc3_4__'></a>

### Review logged test results

Now let's take a look at the results that were logged to the ValidMind Platform:

1. From the **Inventory** in the ValidMind Platform, go to the model you registered earlier.

2. In the left sidebar that appears for your model, click **Documentation** under Documents.

3. Click on any section heading to expand that section to add a new test-driven block ([Need more help?](https://docs.validmind.ai/developer/model-documentation/work-with-test-results.html)).

4. Under TEST-DRIVEN in the sidebar, click **Custom**.

5. Confirm that you're able to insert the following logged results:

    - `pii_demo.PIIDetection:disabled`
    - `pii_demo.PIIDetection:desc_blocked`
    - `pii_demo.PIIDetection:override_results`
    - `pii_demo.PIIDetection:override_both`

<a id='toc4__'></a>

## Troubleshooting

- [x] If you see warnings that Presidio or Presidio analyzer is unavailable, ensure you installed extras: `validmind[pii-detection]`.
- [x] Ensure your environment is restarted after installing new packages if imports fail.

<a id='toc5__'></a>

## Learn more

We offer many interactive notebooks to help you document models:

- [Run tests & test suites](https://docs.validmind.ai/guide/testing-overview.html)
- [Code samples](https://docs.validmind.ai/guide/samples-jupyter-notebooks.html)

Or, visit our [documentation](https://docs.validmind.ai/) to learn more about ValidMind.